In [0]:
from pyspark.sql.functions import first, rand, count
dates = spark.sql("SELECT explode(sequence(DATE'2024-01-01', DATE'2024-03-24', INTERVAL 1 DAY)) as  calendar_date")
c_id = spark.sql("SELECT explode(sequence(1,200, 1)) as  client_id")
types = spark.sql("""SELECT concat("col_", colName) as col_name from (SELECT explode(sequence(1,20, 1)) as  colName)""")
 
dates = dates.repartition(99)
c_id = c_id.repartition(11)
types = types.repartition(1)
 
df_cartesian = c_id.crossJoin(dates.select("calendar_date")).crossJoin(types.select("col_name")).select("client_id","calendar_date","col_name")
df_cartesian2 = df_cartesian.groupBy("calendar_date").agg(count("client_id"))
 
# display(df_cartesian2.limit(1000))
 
df_cartesian = df_cartesian.withColumn("val", (rand()*10).cast("int"))

df_grp = df_cartesian.groupBy("client_id","calendar_date").pivot("col_name").agg((first("val").alias("val")))

display(df_grp.limit(10))


client_id,calendar_date,col_1,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_2,col_20,col_3,col_4,col_5,col_6,col_7,col_8,col_9
170,2024-03-19,6,0,5,6,2,2,3,8,5,2,5,3,9,1,7,3,0,5,5,8
170,2024-03-07,6,7,2,3,7,9,3,2,7,3,3,9,4,8,9,1,6,8,3,5
170,2024-01-17,5,3,2,9,8,4,2,7,3,5,1,6,4,1,6,4,7,4,5,8
170,2024-01-31,9,4,9,9,9,7,8,9,3,7,0,4,7,0,7,0,8,7,0,5
170,2024-02-26,0,2,8,1,7,6,7,1,6,4,6,5,7,0,3,5,4,4,9,8
170,2024-03-21,2,5,3,7,3,9,8,6,9,9,5,3,7,9,8,5,4,7,4,2
170,2024-01-29,2,9,0,6,8,2,3,3,9,4,4,1,3,7,9,7,3,0,0,8
170,2024-03-24,5,6,5,4,5,4,6,3,3,5,1,1,1,8,8,4,3,7,2,9
170,2024-02-24,5,0,2,0,7,1,1,4,8,6,0,2,9,9,2,2,9,6,8,5
170,2024-02-29,5,0,2,6,3,8,3,2,0,0,3,6,2,0,6,0,2,0,9,6


In [0]:
# zduplikowana nazwa kolumny
df_cartesian2_renamed = df_cartesian2.withColumnRenamed("calendar_date", "calendar_date_right")

# inner
inner_join_result = df_cartesian.join(df_cartesian2_renamed, df_cartesian["calendar_date"] == df_cartesian2_renamed["calendar_date_right"], "inner")

# Usuwanie duplikatów
inner_without_duplicates = inner_join_result.drop("calendar_date_right")

# Wyświetlenie wyników złączenia
display(inner_without_duplicates.limit(10))

client_id,calendar_date,col_name,val,count(client_id)
170,2024-03-19,col_1,6,4000
170,2024-03-19,col_2,6,4000
170,2024-03-19,col_3,5,4000
170,2024-03-19,col_4,9,4000
170,2024-03-19,col_5,0,4000
170,2024-03-19,col_6,2,4000
170,2024-03-19,col_7,2,4000
170,2024-03-19,col_8,5,4000
170,2024-03-19,col_9,5,4000
170,2024-03-19,col_10,5,4000


In [0]:
# left
left_join_result = df_cartesian.join(df_cartesian2, df_cartesian["calendar_date"] == df_cartesian2["calendar_date"], "left")

# drop "calendar_date"
left_without_duplicates = left_join_result.drop(df_cartesian2["calendar_date"])

display(left_without_duplicates.limit(10))

client_id,col_name,val,calendar_date,count(client_id)
170,col_1,6,2024-03-19,4000
170,col_2,6,2024-03-19,4000
170,col_3,5,2024-03-19,4000
170,col_4,9,2024-03-19,4000
170,col_5,0,2024-03-19,4000
170,col_6,2,2024-03-19,4000
170,col_7,2,2024-03-19,4000
170,col_8,5,2024-03-19,4000
170,col_9,5,2024-03-19,4000
170,col_10,5,2024-03-19,4000
